In [13]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import warnings

In [15]:
warnings.filterwarnings('ignore')

In [17]:
SCORE_DIR = "data2/scores" 

In [19]:
box_scores = os.listdir(SCORE_DIR)

In [21]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [23]:
def parse_html(box_score):
    with open(box_score, "r+", encoding='latin-1') as f:

        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup 

In [25]:
def read_line_score(soup):
    try:
        line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
        cols = list(line_score.columns)
        cols[0] = "team"
        cols[-1] = "total"
        line_score.columns = cols
        line_score = line_score[["team", "total"]]
        return line_score
    except (ValueError, IndexError) as e:
        print(f"Error reading line score table: {e}")
        # Return empty DataFrame or None as fallback
        return pd.DataFrame(columns=["team", "total"])
    except Exception as e:
        print(f"Unexpected error: {e}")
        return pd.DataFrame(columns=["team", "total"])

In [27]:
def read_stats(soup, team, stat):
    try:
        tables = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)
        if not tables:
            return pd.DataFrame()  # Return empty DataFrame if no tables found
        df = tables[0]
        df = df.apply(pd.to_numeric, errors="coerce")
        return df
    except (ValueError, IndexError):
        # Handle cases where table doesn't exist or is empty
        return pd.DataFrame()

In [29]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs= [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [33]:
games = []

base_cols = None
for i, box_score in enumerate(box_scores):  # Fixed: changed box_score to box_scores
    try:
        soup = parse_html(box_score)
        
        # Check if soup is None (parsing failed)
        if soup is None:
            print(f"Skipping - parsing failed: {box_score}")
            continue
            
        line_score = read_line_score(soup)
        if line_score is None or line_score.empty:
            print(f"Skipping - no line score: {box_score}")
            continue
            
        teams = list(line_score["team"])
        if len(teams) != 2:
            print(f"Skipping - expected 2 teams, got {len(teams)}: {box_score}")
            continue
            
        summaries = [] 
        for team in teams:
            basic = read_stats(soup, team, "basic")
            advanced = read_stats(soup, team, "advanced")
            
            # Skip if stats are empty
            if basic.empty or advanced.empty:
                print(f"Skipping team {team} - missing stats: {box_score}")
                break  # break out of team loop, this game is invalid
                
            totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
            totals.index = totals.index.str.lower() 
            
            maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
            maxes.index = maxes.index.str.lower() + "_max"
        
            summary = pd.concat([totals, maxes])
            if base_cols is None:
                base_cols = list(summary.index.drop_duplicates(keep="first"))
                base_cols = [b for b in base_cols if "bpm" not in b]
        
            summary = summary[base_cols]
            summaries.append(summary)
        
        # Check if we successfully processed both teams
        if len(summaries) != 2:
            print(f"Skipping - incomplete team data: {box_score}")
            continue
            
        summary = pd.concat(summaries, axis=1).T
        
        game = pd.concat([summary, line_score], axis=1)
        game["home"] = [0, 1]
        
        game_opp = game.iloc[::-1].reset_index()
        game_opp.columns += "_opp"
        
        full_game = pd.concat([game, game_opp], axis=1)
        
        full_game["season"] = read_season_info(soup)
        full_game["date"] = os.path.basename(box_score)[:8]
        full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
        full_game["won"] = full_game["total"] > full_game["total_opp"]
        games.append(full_game)

        if len(games) % 100 == 0:
            print(f"{len(games)} / {len(box_scores)}")
            
    except Exception as e:
        print(f"Error processing {box_score}: {str(e)}")
        continue

100 / 12846
200 / 12846
300 / 12846
400 / 12846
500 / 12846
600 / 12846
700 / 12846
800 / 12846
900 / 12846
1000 / 12846
1100 / 12846
Unexpected error: Document is empty
Skipping - no line score: data2/scores\201604010MEM.html
Unexpected error: Document is empty
Skipping - no line score: data2/scores\201604010MIL.html
Unexpected error: Document is empty
Skipping - no line score: data2/scores\201604010NYK.html
Unexpected error: Document is empty
Skipping - no line score: data2/scores\201604010UTA.html
1200 / 12846
1300 / 12846
1400 / 12846
1500 / 12846
1600 / 12846
1700 / 12846
1800 / 12846
1900 / 12846
2000 / 12846
2100 / 12846
2200 / 12846
2300 / 12846
2400 / 12846
2500 / 12846
2600 / 12846
2700 / 12846
2800 / 12846
2900 / 12846
3000 / 12846
3100 / 12846
3200 / 12846
3300 / 12846
3400 / 12846
3500 / 12846
3600 / 12846
3700 / 12846
3800 / 12846
3900 / 12846
4000 / 12846
4100 / 12846
4200 / 12846
4300 / 12846
4400 / 12846
4500 / 12846
4600 / 12846
4700 / 12846
4800 / 12846
4900 / 12846


In [35]:
games_df = pd.concat(games, ignore_index=True)

In [37]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,240.0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25679,240.0,240.0,40.0,94.0,0.426,14.0,32.0,0.438,26.0,32.0,...,100.0,32.2,230.0,136.0,IND,109,0,2025,2025-06-16,True
25680,240.0,240.0,31.0,74.0,0.419,8.0,30.0,0.267,21.0,26.0,...,100.0,28.4,177.0,102.0,IND,108,1,2025,2025-06-19,False
25681,240.0,240.0,38.0,92.0,0.413,15.0,42.0,0.357,17.0,25.0,...,38.1,44.1,186.0,114.0,OKC,91,0,2025,2025-06-19,True
25682,240.0,240.0,29.0,70.0,0.414,11.0,28.0,0.393,22.0,29.0,...,100.0,82.8,162.0,114.0,OKC,103,1,2025,2025-06-22,False


In [41]:
games_df.to_csv("NBA_games.csv")